# IoT Data Ingestion
Load JSON files from Volume into Delta Table


In [2]:
%run ./data_store_variables


📋 DATA CONFIGURATION VARIABLES
Project Name:        Telecom_IoT_Dashboard
Databricks User:     kunal.gaurav@databricks.com
Clean Username:      kunal-gaurav
Catalog:             kunal
Schema:              telcom
Volume Name:         raw_data
Volume Path:         /Volumes/kunal/telcom/raw_data
Table Name:          kunal.telcom.iot_data_synched_cont


NameError: name 'FULL_TABLE_NAME' is not defined

NameError: name 'FULL_TABLE_NAME' is not defined

In [ ]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, DoubleType, IntegerType, TimestampType

schema = StructType() \
    .add("tower_id", StringType()) \
    .add("region", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("data_usage_mb", DoubleType()) \
    .add("call_drop_rate", DoubleType()) \
    .add("active_users", IntegerType())

# Volume path for IOT logs
volume_path = f"{VOLUME_PATH}/IOT_LOGS"

# Checkpoint location
checkpoint_path = f"{VOLUME_PATH}/checkpoint"

# Read from JSON files (simulated streaming)
stream_df = (spark.readStream
    .schema(schema)
    .json(volume_path))

query = (stream_df
 .writeStream
 .format("delta")
 .outputMode("append")
 .option("checkpointLocation", checkpoint_path)
 .trigger(processingTime="10 seconds")
 .queryName("iot_data_streaming")  # Named query
 .table(TABLE_NAME)
)

print(f"✅ Streaming started: {query.name}")
print(f"📊 Query ID: {query.id}")

In [ ]:
# Get query stats
query_status = query.status

print("=" * 60)
print("📊 STREAMING QUERY STATUS")
print("=" * 60)
print(f"Query Name: {query.name}")
print(f"Query ID: {query.id}")
print(f"Is Active: {query.isActive}")
print(f"Message: {query_status['message']}")
print(f"Data Available: {query_status['isDataAvailable']}")
print(f"Trigger Active: {query_status['isTriggerActive']}")
print("=" * 60)

# Get recent progress
recent_progress = query.recentProgress
if recent_progress:
    latest = recent_progress[-1]
    print("\n📈 LATEST PROGRESS:")
    print(f"Batch ID: {latest.get('batchId', 'N/A')}")
    print(f"Input Rows: {latest.get('numInputRows', 0)}")
    print(f"Processing Time: {latest.get('durationMs', {}).get('triggerExecution', 'N/A')} ms")
    print(f"Timestamp: {latest.get('timestamp', 'N/A')}")

In [ ]:
# Stop the streaming query
try:
    query.stop()
    print(f"⏹️  Streaming query '{query.name}' stopped successfully")
    print(f"   Final status: {query.isActive}")
except Exception as e:
    print(f"❌ Error stopping query: {e}")

# Verify all streaming queries are stopped
active_queries = spark.streams.active
if active_queries:
    print(f"\n⚠️  Still {len(active_queries)} active streaming queries:")
    for q in active_queries:
        print(f"   - {q.name} (ID: {q.id})")
else:
    print("\n✅ No active streaming queries")